In [1]:
import torch.nn as nn
import torch
import torchvision
from sklearn.model_selection import train_test_split
from torchvision import models
from torchvision.models import efficientnet_v2_s
from torchvision.transforms import v2
from torchvision import datasets
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import StepLR
import numpy as np

#PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 # mps de memory overflow hatası alıyorsanız bu satırı ekleyin colabde mps yok


In [2]:
data_dir = "dataset/" # colab de data_dir = "drive/MyDrive/dataset/"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
    # mps is for apple
else:
    device = torch.device("cpu")
    # if you use cpu probably you will get memory error or it will take too long time

transform = v2.Compose([
    v2.Resize((224, 224)),
    #v2.RandomHorizontalFlip(p=0.5),  # Yatay çevirme
    #v2.RandomRotation(degrees=5),  # Hafif döndürme
    #v2.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),  # Renk artırma
    #v2.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 2.0)),  # Gaussian bulanıklık
    #v2.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),  # Rastgele kırpma
    #v2.RandomErasing(p=0.3, scale=(0.02, 0.1), ratio=(0.3, 3.3)),  # Rastgele silme
    v2.ToImage(),
    v2.ToDtype(torch.bfloat16, scale=True),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder(data_dir, transform=transform)
train_dataset, test_dataset = train_test_split(dataset, test_size=0.33, stratify=dataset.targets)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# burada classlar arasında eşitsizlik olduğundan classlar arasında dengesizlik olabilir. Bu durumda classlar arasında dengesizliği gidermek için cross entropy loss fonksiyonuna class_weights parametresi ekledim.
class_counts = np.bincount(dataset.targets)
class_weights = 1.0 / torch.tensor(class_counts, dtype=torch.bfloat16)

In [3]:
def train(model, train_loader, test_loader, num_epochs=20):
    loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Her *** epoch'ta LR %10'a düşer overfittin ihtimalini azaltmak için yinede cok random bir parametre olmasada olur

    total_accuracy = 0

    # overfitting i engellemek için en iyi modeli seciyor
    best_model = {"model": None, "accuracy": 0, "test_loss": float("inf")}
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        train_loss = 0.0

        # Training loop
        for inputs, labels in tqdm(train_loader, desc="Training Progress"):

            #inputs = inputs.view(inputs.size(0), -1) # Flatten the input data

            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Evaluation
        model.eval()  # Set model to evaluation mode
        test_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad(): # Gradients are not calculated in this block
            for inputs, labels in test_loader:
                #inputs = inputs.view(inputs.size(0), -1)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = loss_fn(outputs, labels)
                test_loss += loss.item()

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        train_loss /= len(train_loader)  # Average training loss
        test_loss /= len(test_loader)  # Average test loss
        accuracy = 100 * correct / total  # Test accuracy
        total_accuracy += accuracy
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:}, Test Loss: {test_loss:}, Accuracy: {accuracy:}%")
        scheduler.step()
        if accuracy > best_model["accuracy"]:
            if test_loss < best_model["test_loss"]:
                best_model["model"] = model
                best_model["accuracy"] = accuracy
                best_model["test_loss"] = test_loss
    print(f"Best model accuracy: {best_model['accuracy']}")

    return best_model["model"]

In [4]:
from torchvision.models import EfficientNet_V2_S_Weights
model = models.efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.DEFAULT).to(device)

model.classifier[1] = nn.Linear(model.classifier[1].in_features, 3).to(device) #son 1000 olan katmanı 3 yaptık çünkü 3 sınıf var
model = model.to(torch.bfloat16)
model = train(model, train_dataloader, test_dataloader, num_epochs=20) # orijinal paperda 20 epoch kullanılmış bu yüzden max 20 yaptım

Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/20, Training Loss: 0.8703125, Test Loss: 1.5390625, Accuracy: 34.285714285714285%


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 2/20, Training Loss: 0.566015625, Test Loss: 2.0, Accuracy: 51.42857142857143%


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

...

Epoch 18/20, Training Loss: 0.06396484375, Test Loss: 0.3878580729166667, Accuracy: 91.42857142857143%


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 19/20, Training Loss: 0.04095458984375, Test Loss: 0.38671875, Accuracy: 91.42857142857143%


Training Progress:   0%|          | 0/5 [00:00<?, ?it/s]

Epoch 20/20, Training Loss: 0.1075927734375, Test Loss: 0.4142252604166667, Accuracy: 91.42857142857143%
Best model accuracy: 92.85714285714286


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score

# Modelin çıktılarını ve etiketlerini çıkartma
def extract_features_and_labels(loader, model):
    model.eval()
    features, labels = [], []
    with torch.no_grad():
        for images, lbls in loader:
            images = images.to(device)
            #images = images.view(images.size(0), -1)
            outputs = model(images)
            #outputs = outputs.view(outputs.size(0), -1)
            outputs = outputs.to(torch.float32)
            features.append(outputs.cpu().numpy())
            labels.append(lbls.numpy())
    return np.vstack(features), np.hstack(labels)

train_features, train_labels = extract_features_and_labels(train_dataloader, model)
test_features, test_labels = extract_features_and_labels(test_dataloader, model)

In [7]:
# SVM eğitimi
svm = SVC(kernel='linear')
svm.fit(train_features, train_labels)

# Test verilerinde değerlendirme
predictions = svm.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
kappa = cohen_kappa_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions, average='weighted')

print(f"Accuracy: {accuracy:}, Kappa: {kappa:}, F1-score: {f1:}")

Accuracy: 0.9285714285714286, Kappa: 0.8856582816073179, F1-score: 0.9286416286416288
